# 1.데이터 다운로드 및 전처리

## 1) pykrx 모듈 다운로드

In [20]:
!pip install pykrx

## 2) 데이터 불러오기 (5년 치, 10년 치)

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pykrx import stock

# KOSDAQ 지수
# df = stock.get_index_ohlcv_by_date("20100101", "20211231", "1001")

# KB금융
df = stock.get_market_ohlcv("20180101", "20221231", "105560")

df_5years = df
df_10years = stock.get_market_ohlcv("20130101", "20221231", "105560")

## 3) 데이터 정규화(Normalization)

In [19]:
df.shape

(1232, 6)

### (1) 5y, 5y-차분

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# graph 1
series_5years = df_5years['종가']
plt.title("5 years")
plt.plot(series_5years)
plt.show()

# graph 2
plt.title("5 years, difference")
series_5years_diff = series_5years - series_5years.shift(1)
plt.plot(series_5years_diff)
plt.figure(figsize=(1,8))
plt.show()

### (2) 10y, 10y differencing

In [ ]:
# graph 3
plt.title("10 years")
series_10years = df_10years['종가']
plt.plot(series_10years)
plt.show()

# graph 4
plt.title("10 years, difference")
series_10years_diff = series_10years - series_10years.shift(1)
plt.plot(series_10years_diff)
plt.show()

### (3) min-max 정규화

In [4]:
from sklearn.preprocessing import MinMaxScaler

min_price = min(df['종가'])
max_price = max(df['종가'])

min_max_scaler = MinMaxScaler()
df["종가"] = min_max_scaler.fit_transform(df["종가"].to_numpy().reshape(-1,1))

In [ ]:
df['종가']

# 2.Dataset 윈도우

In [5]:

from torch.utils.data import DataLoader, Dataset
class windowDataset(Dataset):
  # data_stream     : input_window, output_window 크기에 따라 쪼개질 데이터
  # input_window    : 인풋 기간
  # output_window   : 아웃풋 기간
  # stride          :
    def __init__(self, data_stream, input_window=80, output_window=20, stride=5):
        # data_stream의 행 개수를 구한다.
        L = data_stream.shape[0]
        # stride에 따라 샘플 개수를 구한다.
        num_samples = (L - input_window - output_window) // stride + 1

        # [window 크기 * sample 개수] 크기의, 0으로 채워진 배열을 만든다.
        X = np.zeros([input_window, num_samples])
        Y = np.zeros([output_window, num_samples])

        # np.arange(num_samples): range(num_samples) 와 같음
        for i in np.arange(num_samples):
            # 1) X:   input_window 만큼 자르기 (stride * i ~)
            start_x = stride * i
            X[:,i] = data_stream[start_x:start_x + input_window]
            # 2) Y:   output_window 만큼 자르기 (stride * i + input_window ~)
            start_y = start_x + input_window
            Y[:,i] = data_stream[start_y:start_y + output_window]


        # shape       : [window 크기, sample 개수]
        X = X.reshape(X.shape[0], X.shape[1], 1).transpose((1,0,2))
        Y = Y.reshape(Y.shape[0], Y.shape[1], 1).transpose((1,0,2))
        self.x = X
        self.y = Y

        self.len = len(X)

    def __getitem__(self, i):
        return self.x[i], self.y[i]
    def __len__(self):
        return self.len



# 3.Transformer 모델

In [48]:

from torch import nn
import torch
import math

class TFModel(nn.Module):

# iw/ow:      input window, output window
# d_model:    인풋 개수
# nlayers:    인코더 부분의 인코더 개수
# nhead:      multihead attention 개수

    def __init__(self, iw: int, ow: int, d_model: int, nhead: int, nlayers: int, dropout=0.5):
        super(TFModel, self).__init__()

        # 1개 인코더, 인풋 사이즈가 d_model이고 attention 개수는 nhead
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout)

        # stacked 인코더, nlayers 만큼 쌓여있다.
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=nlayers)
        self.pos_encoder = PositionalEncoding(d_model, dropout)

        # 인풋 차원 변환. 1차원 -> d_model//2차워 -> d_model차원
        self.encoder = nn.Sequential(
            nn.Linear(1, d_model//2),
            nn.ReLU(),
            nn.Linear(d_model//2, d_model)
        )

        # 차원 변환. d_model -> d_model//2 -> 1
        self.linear =  nn.Sequential(
            nn.Linear(d_model, d_model//2),
            nn.ReLU(),
            nn.Linear(d_model//2, 1)
        )

        # 차원 변환. iw -> ow
        self.linear2 = nn.Sequential(
            nn.Linear(iw, (iw+ow)//2),
            nn.ReLU(),
            nn.Linear((iw+ow)//2, ow)
        )

    def generate_square_subsequent_mask(self, size):
        mask = (torch.triu(torch.ones(size, size)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src, srcmask):
        src = self.encoder(src)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src.transpose(0,1), srcmask).transpose(0,1)
        output = self.linear(output)[:,:,0]
        output = self.linear2(output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

def gen_attention_mask(x):
    mask = torch.eq(x, 0)
    return mask


# 4.학습

## 1) input, output 윈도우 사이즈 설정

In [60]:
INPUT_WINDOW = 24*14
OUTPUT_WINDOW = 24

data_train = series_5years
train_dataset = windowDataset(data_train, input_window=INPUT_WINDOW, output_window=OUTPUT_WINDOW, stride=1)
train_loader = DataLoader(train_dataset, batch_size=64)     # 64 = 2^6, 512 = 2^9

if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

lr = 1e-4
model = TFModel(iw=INPUT_WINDOW, ow=OUTPUT_WINDOW, d_model=512, nhead=8, nlayers=4, dropout=0.1).to(device)
criterion = nn.MSELoss()                                            # MSEloss(): ow 각 요소들의 합
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [61]:
data_train.head()

날짜
2018-01-02    63100
2018-01-03    63100
2018-01-04    63000
2018-01-05    64100
2018-01-08    66600
Name: 종가, dtype: int32

In [ ]:
'''
[error]
OutOfMemoryError: CUDA out of memory. Tried to allocate 126.00 MiB (GPU 0; 14.75 GiB total capacity; 13.76 GiB already allocated; 70.81 MiB free; 13.91 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

[solution]
https://discuss.pytorch.kr/t/cuda-out-of-memory/216/6
'''
import torch, gc
gc.collect()
torch.cuda.empty_cache()

# for tqdm
from tqdm import tqdm
epoch = 30
model.train()
progress = tqdm(range(epoch))

for i in progress:
    batchloss = 0.0
    for (inputs, outputs) in train_loader:
      # inputs.shape: [batch_size, iw, 1]
      # outputs.shape: [batch_size, ow, 1]

      # Initialize grad
      optimizer.zero_grad()                                           # zero_grad()로 Torch.Tensor.grad 초기화. 초기화하지 않으면 다음 루프 backward() 시에 간섭함.

      # Forward propagation with masking
      src_mask = model.generate_square_subsequent_mask(inputs.shape[1]).to(device)
      result = model(inputs.float().to(device), src_mask)             # forward

      # Backward propagation
      loss = criterion(result, outputs[:,:,0].float().to(device))     # ?? 64개 중 하나만 loss를 담네?
      loss.backward()                                                 # backward
      optimizer.step()
      batchloss += loss
    progress.set_description("loss: {:0.6f}".format(batchloss.cpu().item() / len(train_loader)))